In [37]:
import sys
sys.path.append("..")
from Package.DQuadratization import *
from Package.EquationSystem import *
from Package.Combinations import *
from qbee import *
import sympy as sp
from Package.Simulation.numerical import *
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

### Application to reachability analysis - Duffing system

---

Reachability problem is a problem, given an ODE system $\mathbf{x}' = \mathbf{p}(\mathbf{x})$, a set $S\subseteq \mathbb{R}^n$ of possible initial conditions, and a time $t \in \mathbb{R}_{>0}$, compute a set containing the set 
$$
\{\mathbf{x}(t) \mid \mathbf{x}'=\mathbf{p}(\mathbf{x}) \;\&\; \mathbf{x}(0) \in S\} \subseteq \mathbb{R}^n
$$
of all points reachable from $S$ at time $t$.
One recent approach to this problem in the vicinity of a dissipative equilibrium $\mathbf{x}^\ast$ proposed by Forets and Schilling in [Forets_2021] is to use Carleman linearization to reduce the problem to the linear case which is well-studied.
However, the approach described in [Forets_2021] is applicable only to quadratic systems. 
Algorithm 3 allows to lift this restriction by computing a quadratization which preserves the dissipativity of $\mathbf{x}^\ast$.

We will illustrate this idea using the **Duffing** equation
$$
  x'' = kx + ax^{3} + bx'
$$
which described damped oscillator with non-linear restoring force.
The equation can be written as a first-order system by introducing $x_1 := x, x_2 := x'$ as follows
$$
\begin{cases}
    x_1' = x_2,
    x_2' = kx_1 + a x_1^3 + b x_2.
\end{cases}
$$

### Inner-quadratic quadraticization

---

In [38]:
x1, x2 = sp.symbols('x1 x2')
a, k, b = sp.symbols('a k b')

duffing_system = [
    sp.Eq(x1, x2),
    sp.Eq(x2, k * x1 + a * x1 ** 3 + b * x2)
]

duffing_eq_system = EquationSystem(duffing_system)

In [39]:
display(duffing_eq_system.show_system_latex())

<IPython.core.display.Latex object>

In [40]:
inner_quadratic_duffing = optimal_inner_quadratization(duffing_eq_system)

The Original System is: 


<IPython.core.display.Latex object>

The Optimal Dissipative Quadratization is: 


<IPython.core.display.Latex object>

The new introduced variables are: 


<IPython.core.display.Latex object>

The Optimal Quadratic Dissipative System is (with substitution): 


<IPython.core.display.Latex object>

### Dissipative Quadratization

---

We have the Jacobian matrix of the duffing system:
$$
J = \begin{bmatrix}
    0 & 1 \\
    k + 3ax_1^2 & b
\end{bmatrix}
$$
and the equilibrium point $\mathbf{x}^\ast = (0, 0), (-\sqrt{-\frac{k}{a}}, 0), (\sqrt{-\frac{k}{a}}, 0)$. Therefore, we have the value for $x_{1}^{2} = - \frac{k}{a}$ and the Jacobian matrix at the equilibrium point $(-\sqrt{-\frac{k}{a}}, 0), (\sqrt{-\frac{k}{a}}, 0)$ is:
$$
J = \begin{bmatrix}
    0 & 1 \\
    -2k & b
\end{bmatrix}
$$
The characteristic polynomial of $J$ is $x^{2}-bx+2k=0$. In order to have all eigenvalues with negative real part, we need to have $b<0$ and $k>0$ by Routh-Hurwitz criterion. Let's take the value $a=-1, b=-1, k=1$. Then we got equilibrium point $\mathbf{x}^\ast = (0, 0), (-1, 0), (1, 0)$ and dissipative equilibrium point $\mathbf{x}^\ast = (-1, 0), (1, 0)$.

In [41]:
duffing_system_1 = [
    sp.Eq(x1, x2),
    sp.Eq(x2, x1 - x1 ** 3 - x2)
]

duffing_eq_system_1 = EquationSystem(duffing_system_1)

#### dissiaptive quadratization with one equilibrium point

In [42]:
# Here we test at equilibrium point (1, 0)
duffing_eq_system_1_one_equilibrium = dquadratization_one_equilibrium(duffing_eq_system_1, {x1: 1, x2: 0}, display=True)

-------------------------- Compute a quadratization dissipative at a given equilibrium --------------------------
The system before coordinate transformation is: 


<IPython.core.display.Latex object>

-------------------------- Dissipative Quadratization on transformed system --------------------------
The Original System is: 


<IPython.core.display.Latex object>

The Optimal Dissipative Quadratization is: 


...

<IPython.core.display.Latex object>

------------------------------ Optimal Dissipative Quadratization ------------------------------
The eigenvalue with the largest real part is (real part):  -0.5
Mode: Default diagonal value, we will choose the largest eigenvalue as the diagonal value.
The converted Optimal Dissipative Quadratization System is: 


<IPython.core.display.Latex object>

The matrix  associated to the linear part system is:


<IPython.core.display.Latex object>

#### dissiaptive quadratization with multiple equilibrium point

In [44]:
# Here we test at equilibrium point (1, 0) (-1, 0)
duffing_eq_system_1_multi_equilibrium = dquadratization_multi_equilibrium(duffing_eq_system_1, [[1,0], [-1,0]], display=True)

-------------------------- Compute a quadratization dissipative with multi given equilibrium --------------------------
The original system is: 


<IPython.core.display.Latex object>

-------------------------- The Dissipative quadratized system --------------------------


<IPython.core.display.Latex object>

-------------------------- The lambda value --------------------------
The lambda value is:  1
-------------------------- The Dissipative quadratized system with lambda value --------------------------


<IPython.core.display.Latex object>

-------------------------- The Jacobian matrix --------------------------


<IPython.core.display.Latex object>